<a href="https://colab.research.google.com/github/mueller-florian-rh/menu-extraction/blob/main/catter_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import getpass
import os
import uuid
import pymupdf
import requests
import csv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)
prompt = PromptTemplate(
    template=open("prompt_speisekarten_extraction.txt", encoding="utf-8").read(),
    input_variables=["full_text"]
)

chain = prompt | llm

print("Enter URL:")
url = input()
tmp_pdf = f"temp_{uuid.uuid4().hex}.pdf"
r = requests.get(url)
with open(tmp_pdf, "wb") as f:
  f.write(r.content)

doc = pymupdf.open(tmp_pdf)
full_text = "\n".join([page.get_text() for page in doc])

result = chain.invoke({"pdf_text" : full_text})

tmp_csv = f"output_{uuid.uuid4().hex}.csv"
with open(tmp_csv, "w", encoding="utf-8", newline="") as f:
  writer = csv.writer(f, delimiter=";")
  writer.writerow(["Kategorie", "Speise", "Beschreibung", "Confidence"])
  for line in result.content.strip().split("\n"):
    parts = line.split(";")
    if len(parts) == 4:
      writer.writerow(parts)

os.remove(tmp_pdf)

Enter URL:
https://www.kaiserwetter-catering.de/wp-content/uploads/KAISERWETTER_Cateringkarte.pdf


In [10]:
!pip install pymupdf uuid

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.8 MB/s eta 0:00:00
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6477 sha256=8a533a73784bd7eb68940261e423a37112aff241323c215b57b06feb550eea5d
  Stored in directory: /root/.cache/pip/wheels/e0/01/df/bd20df409bd81f8b99e6cd343c5f49731dc0a20eefefdafae0
Successfully built uuid
